In [7]:
import numpy as np
import cv2 as cv
import mediapipe as mp
import math

In [8]:
mp_face_mesh = mp.solutions.face_mesh
avg_dist = []

In [9]:
LEFT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]
RIGHT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_IRIS = [469, 470, 471, 472]
RIGHT_IRIS = [474, 475, 476, 477]
# CENTERS = [468,473]
LEFT_CENTER = [468]
RIGHT_CENTER = [473]

In [10]:
def mediapipe_detection(image, face):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = face.process(image)
    image.flags.writeable = True
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
    return image, results

In [11]:
def part_detection(part, facial_landmarks):
    idx_to_coordinates = {}
    for idx, landmark in enumerate(part):
        coord = facial_landmarks.landmark[landmark]
        p_x = min(math.floor(coord.x * img_height), img_height - 1)
        p_y = min(math.floor(coord.y * img_width), img_width - 1)
        landmark_coord = (p_x, p_y)

        if landmark_coord:
            idx_to_coordinates[idx] = landmark_coord
    return idx_to_coordinates

In [12]:
def landmark_distance(image, coord1, coord2):
    
    cv.line(image, coord1[0], coord2[0], (0,255,0), 2)
    irises_dist_in_pixels = math.dist(coord1[0], coord2[0])
    irises_dist_in_rw = 7
    face_dist_in_rw = 80
    focal_length = (irises_dist_in_pixels * face_dist_in_rw) / irises_dist_in_rw
    avg_dist.append(focal_length)
    
    
    #focal length for distance = 70cm is 611cm
    #focal length for distance = 60cm is 618cm
    #focal length for distance = 50cm is 602cm
    #focal length for distance = 40cm is 610cm
    #average focal length = 610.25cm
    avg_focal_length = 610
    actual_distance = (avg_focal_length * irises_dist_in_rw) / irises_dist_in_pixels
    return focal_length, actual_distance

In [13]:
def display_part(image, coords, color, show_landmarks = False):
    points = np.zeros(shape = (len(coords), 2))
    for idx, landmark_px in coords.items():
        points[idx] = landmark_px
        if show_landmarks == True:
            cv.circle(image, landmark_px, 1, (255, 0, 0), 2)
    points = points.astype(np.int32)
#     print(points)

    cv.polylines(image, [points], True, color, 2)

In [15]:
cap = cv.VideoCapture(0)
k = 0
with mp_face_mesh.FaceMesh(
    max_num_faces = 1,
    refine_landmarks = True,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5
) as face_mesh:
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("Not ret")
            break
            
        img_width, img_height, _ = frame.shape
        image = cv.flip(frame, 1)
        image, results = mediapipe_detection(image, face_mesh)

        
        if results.multi_face_landmarks:
            for num, facial_landmarks in enumerate(results.multi_face_landmarks):


                left_centers_coords = part_detection(LEFT_CENTER, facial_landmarks)
                right_centers_coords = part_detection(RIGHT_CENTER, facial_landmarks)
                
                display_part(image, left_centers_coords, (0,255,255), True)
                display_part(image, right_centers_coords, (0,255,255), True)
        
                
                focal_length, actual_distance = landmark_distance(image, left_centers_coords, right_centers_coords)
                
                cv.putText(image, f'dist:{actual_distance} cm', (100, 100), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv.LINE_AA)
        k += 1
        cv.imshow("video feed", image)
        if cv.waitKey(5) & 0xff == ord('q'):
            break

print(sum(avg_dist) / len(avg_dist))
avg_dist = []
cap.release()
cv.destroyAllWindows()


661.0438322947307
